In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

## 2.3 Negotiations, auctions, and optimization
### 2.3.1 From contract nets to auction-like optimization
In a contract net we have a global problem, made up of a number of tasks. For each agent there is a cost function $c_i(T)$ reflecting the cost of agent $i$ to do all the tasks $T$. We start agents out with random tasks, and want them to swap and exchange them in order to lower the sum of the individual costs. In future chapters we will look at how to do this with agents which are also self-interested. First, we derive a linear programming solution to this problem. 
### 2.3.2 The assignment problem and linear programming
In a (symmetric) assignment problem we have a set $N$ of agents, a set $X$ of objects, a set of possible assignment pairs $M$, and a function $v$ which tells us what the value of an assignment pair is. At the moment we just consider that we have at most 1 object per agent. A valid assignment is then a set of pairs $S\subseteq M$, which has the property that each each agent $i$ and object $j$ is in at most 1 pair. A feasable assignment is one in which all agents are assigned an object. An optimal assignment is one in which the sum of the values is maximised. 

This problem can be expressed as a linear program. We introduce an indicator matrix $x$ where $x_{i,j}=1$ if the pair ($i$,$j$) is selected. Then the linear program is:

$$
\begin{align*}
\text{maximise} \quad & \sum_{i,j\in M} v(i,j)x_{i,j} \\
\text{subject to} \quad & \sum_{j} x_{i,j}<=1 \\
\text{} \quad & \sum_{i} x_{i,j}<=1
\end{align*}
$$

While this might seem incorrect, as this would seem to possibly create partial assignments, in truth the answers turn out to be integers. In fact, every linear program encoding of an assignment problem has an integer solution. Solving a linear program can then be done in polynomial time, using the simplex algorithm.

For example, let's consider $N=[\text{john},\text{mary},\text{leon}]$, $X=[\text{cat},\text{dog},\text{hat}]$, and the value function is:

$
\begin{array}{c|ccc}
\text{} & \text{cat} & \text{dog} & \text{hat} \\
\hline
\text{john} & 1 & 3 & 0 \\
\text{mary} & 3 & 1 & 2 \\
\text{leon} & 4 & 1 & 0 \\
\end{array}
$

In [70]:
agents = ["john","mary","leon"]
objects = ["cat","dog","hat"]
value_dict = {
    ("john","cat"):1,("john","dog"):3,("john","hat"):0,
    ("mary","cat"):3,("mary","dog"):1,("mary","hat"):2,
    ("leon","cat"):4,("leon","dog"):1,("leon","hat"):0,
}
value_vector = list(value_dict.values())
x = np.zeros(9)
A = np.array([
    [1,1,1,0,0,0,0,0,0], # john can't be assigned to more than 1 item.
    [0,0,0,1,1,1,0,0,0], # mary
    [0,0,0,0,0,0,1,1,1], # leon
    [1,0,0,1,0,0,1,0,0], # the cat can't be assigned to more than 1 person
    [0,1,0,0,1,0,0,1,0], # the dog
    [0,0,1,0,0,1,0,0,1], # the hat
])
b = np.ones(6) # 1 for each 'only 1' condition
res = linprog(-np.array(value_vector), A_ub=A, b_ub=b, bounds=[(0,None) for _ in range(9)])
print("resulting weights:")
print(res["x"])
print("assignment")
assignment = [list(value_dict.keys())[i] for i in np.where(res["x"]==1)[0]]
print(assignment)

resulting weights:
[ 0.  1.  0.  0.  0.  1.  1. -0.  0.]
assignment
[('john', 'dog'), ('mary', 'hat'), ('leon', 'cat')]


the problem with this is that it isn't particularly parallelizable, and that small changes in the inputs mean the problem has to be redone from scratch. Here we explore an alternative, based on the idea of competitive equilibrium.

#### The assignment problem and competitive equilibrium

Imagine we have a set of prices $p$ for each of the resources (cat, dog, hat). If we define the the utility to each person of an assignment as the value of the assignment minus the price ($v(i,j)-p_j$), then we can define a competitive equilibrium: A situation in which nobody can improve their utility by changing to another good.

Then there is a remarkable theory that if an assignment and a price are in competitive equilibrium the assignment is optimal. We will look at why this is later, but it stems from the dual problem of the previous section.

For now we will look at a simple approach to do assignments in a multiagent manner, using the concept of an auction.

#### A naive auction algorithm

The following algorithm is almost optimal, but has a flaw (which we will fix in the next section). The basic idea is to have each agent bid for the items, increasing the bid by the difference between their first and second favorite option.

The array of values again:

$
\begin{array}{c|ccc}
\text{} & \text{cat} & \text{dog} & \text{hat} \\
\hline
\text{john} & 1 & 3 & 0 \\
\text{mary} & 3 & 1 & 2 \\
\text{leon} & 4 & 1 & 0 \\
\end{array}
$

In [71]:
prices = {"cat":0,"dog":0,"hat":0}
assignments = dict(zip(agents,[None for _ in range(len(agents))]))
for epoch in range(100):
    all_assigned = True
    for agent in agents:
        if(assignments[agent]==None):
            all_assigned = False
            # get the utilities (value - price)
            u = {good:value_dict[(agent,good)]-prices[good] for good in prices.keys()}
            sorted_u_keys = np.array(list(u.keys()))[np.argsort(list(u.values()))[::-1]] # this just sorts, ignore
            favorite = sorted_u_keys[0]
            second_favorite = sorted_u_keys[1]
            # you are willing to pay the difference between your first and second favorite
            increment = u[favorite]-u[second_favorite]
            prices[favorite]=prices[favorite]+increment
            for other_agent in agents:
                if assignments[other_agent]==favorite:
                    assignments[other_agent]=None
            assignments[agent]=favorite
            print(agent,"increments",favorite,"to",prices[favorite])
    if all_assigned:
        break
    print()
    print("prices: ",prices)
    print()
    print("assignments: ",assignments)
    print()

john increments dog to 2
mary increments cat to 1
leon increments cat to 4

prices:  {'cat': 4, 'dog': 2, 'hat': 0}

assignments:  {'john': 'dog', 'mary': None, 'leon': 'cat'}

mary increments hat to 3

prices:  {'cat': 4, 'dog': 2, 'hat': 3}

assignments:  {'john': 'dog', 'mary': 'hat', 'leon': 'cat'}



This produces the final assignments, which are exactly those at a competitive equilibrium.

#### The terminating auction algorithm

While this algorithm works in the above case, it won't work in cases where two values are the same for two different people. In order to deal with that scenario we add a small coefficient $\epsilon$ to the bid increment. While this works it does mean we have to accept some inaccuracy in the outcome. Under certain situations the outcome will still be optimal.

In the next section we will look more closely at the justification of this algorithm.